In [1]:
#Instaling the needed libraries 

!pip install pmdarima
import pandas as pd
import numpy as np
from prophet.plot import plot_plotly, plot_components_plotly
from fbprophet import Prophet
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.arima.model import ARIMA
import statsmodels.api as sm
from pmdarima import auto_arima
import warnings 
from sklearn.metrics import mean_squared_error, mean_absolute_error
warnings.filterwarnings("ignore")

"""
Create time series features based on time series index.
"""

def create_features(df):
   
    df = df.copy()
    df['hour'] = df.index.hour
    df['dayofweek'] = df.index.dayofweek
    df['quarter'] = df.index.quarter
    df['month'] = df.index.month
    df['year'] = df.index.year
    df['dayofyear'] = df.index.dayofyear
    df['dayofmonth'] = df.index.day
    df['weekofyear'] = df.index.isocalendar().week.astype(int)
    return df

'''
Reading and Data preparation  
'''

#read CSV file
df = pd.read_csv('sickness_table.csv')

#set the date as index and converting it to datetime 
df = df.set_index('date')
df.index = pd.to_datetime(df.index)

#dropping empty column 
df.drop('Unnamed: 0', axis=1, inplace=True)

#calculating sickness rate
df['sickness rate']= df['n_sick']/df['n_duty']

#calculating average hundling per call (AHC) and (AHC_target) after concedring dafted
df['AHC']=df['calls']/ (df['n_duty']+df['n_sby']-df['n_sick'])
df['AHC_target']=df['calls']/ (df['n_duty']+df['n_sby']+df['dafted']-df['n_sick'])
df


'''
Cumpute and visualizing Correlation Matrix "heat map"
'''
# Compute the correlation matrix
corr_matrix = df.corr()

# Set up the matplotlib figure with gray background
plt.figure(figsize=(10, 8), facecolor='lightgray')

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt=".2f", linewidths=0.5)

# Add title
plt.title('Correlation Matrix')

# Show plot
plt.show()

 
    
    
'''
 visualizing data distribution
'''

# Define the number of rows and columns for subplots
num_rows = 3
num_cols = 3

# Create subplots
fig, axes = plt.subplots(num_rows, num_cols, figsize=(15, 12))

# Flatten the axes array to easily iterate over each subplot
axes = axes.flatten()

# Define colors
colors = ['tab:blue', 'tab:orange', 'tab:green', 'tab:red', 'tab:purple', 'tab:brown', 'tab:pink', 'tab:gray', 'tab:olive']

# Iterate over each column and plot it on a separate subplot
for i, (col, color) in enumerate(zip(df.columns, colors)):
    ax = axes[i]  # Get the current subplot
    
    # Plot the data
    ax.plot(df.index, df[col], color=color, linestyle='-')
    
    # Set title and labels
    ax.set_title(col)
    ax.set_xlabel('date')
    ax.set_ylabel(col)
    
    # Rotate x-axis labels for better readability
    ax.tick_params(axis='x', rotation=45)
    
    # Remove grid
    ax.grid(False)

# Adjust layout and show plot
plt.tight_layout()
plt.show()

##Create time series features based on time series index
df=create_features(df)
df



'''
Data spliting 
'''
## split the data train data 80% (922), test data 20% (230)
train=df.iloc[:-230]
test=df.iloc[-230:]


'''
creating empty data frame to store the results 
'''

# Define column names
columns = ['Predicted calls Arima', 'Predicted sickness rate Arima', 
           'Predicted calls Prophet', 'Predicted sickness rate Prophet', 
           'Predicted calls XGboost', 'Predicted sickness rate XGboost']

# Create an empty DataFrame with the specified column names
results_df = pd.DataFrame(columns=columns)

# Assuming 'test' is your DataFrame containing the additional columns
test_columns = ['dafted', 'sby_need', 'n_duty']

# Add columns from 'test' DataFrame to the empty DataFrame
for col in test_columns:
    results_df[col] = test[col]

# Set the index of the results_df to match the index of the test DataFrame
results_df.index = test.index

# Display the resulting DataFrame
results_df





'''
ARIMA model function
'''


def arima_model(training_data, testing_data, order, results_df, predicted_column_name, plot_title):
    """
    Apply ARIMA model to training data, test on testing data, and return evaluation metrics and visualization.
    Add predicted values to the provided DataFrame.

    Args:
    - training_data: pandas Series, training data for the ARIMA model
    - testing_data: pandas Series, testing data for evaluating the ARIMA model
    - order: tuple, order of the ARIMA model (p, d, q)
    - results_df: pandas DataFrame, DataFrame to store the results
    - predicted_column_name: str, name of the column to store the predicted values in the results_df
    - plot_title: str, title of the plot

    Returns:
    - evaluation_metrics: dictionary, evaluation metrics including mean squared error, mean absolute error, and mean absolute percentage error
    """

    # Fit ARIMA model
    model = ARIMA(training_data, order=order)
    fitted_model = model.fit()

    # Forecast
    predicted_values = fitted_model.forecast(steps=len(testing_data))

    # Add predicted values to DataFrame
    results_df[predicted_column_name] = predicted_values

    # Evaluation metrics
    mse = mean_squared_error(testing_data, predicted_values)
    mae = mean_absolute_error(testing_data, predicted_values)
    mape = np.mean(np.abs((testing_data - predicted_values) / testing_data)) * 100

    # Visualization
    plt.figure(figsize=(10, 5))
    plt.plot(training_data, label='Training Data')
    plt.plot(testing_data.index, testing_data, label='Actual Values')
    plt.plot(testing_data.index, predicted_values, label='Predicted Values', color='red')
    plt.title(plot_title)
    plt.xlabel('Date')
    plt.ylabel('Value')
    plt.legend()
    plt.show()

    # Return results
    evaluation_metrics = {'Mean Squared Error': mse, 'Mean Absolute Error': mae, 'Mean Absolute Percentage Error': mape}
    return evaluation_metrics


'''
make prediction using ARIMA model function 
'''
# after Performing stepwise search to minimize aic the best order (2,1,1) 
arima_model(train['calls'], test['calls'], (2,1,1), results_df, 'Predicted calls Arima', 'Predicted calls Arima')


'''
make prediction using ARIMA model function 
'''
# after Performing stepwise search to minimize aic the best order (1,1,1) 
arima_model(train['sickness rate'], test['sickness rate'], (1,1,1), results_df, 'Predicted sickness rate Arima', 'Predicted sickness rate Arima')

'''
create function for XGboost model
'''

def xgboost_model(training_data, testing_data, target_column, results_df, predicted_column_name, plot_title):
    """
    Apply XGBoost model to training data, test on testing data, and add predicted values to results DataFrame.

    Args:
    - training_data: pandas DataFrame, training data for the XGBoost model with features and target variable
    - testing_data: pandas DataFrame, testing data for evaluating the XGBoost model with features and target variable
    - target_column: str, name of the target variable column
    - results_df: pandas DataFrame, DataFrame to store the results
    - predicted_column_name: str, name of the column to store the predicted values in the results_df
    - plot_title: str, title of the plot

    Returns:
    - evaluation_metrics: dictionary, evaluation metrics including mean squared error, mean absolute error, and mean absolute percentage error
    """

    # Extract features and target variable from training data
    X_train = training_data.drop(columns=[target_column])
    y_train = training_data[target_column]

    # Extract features and target variable from testing data
    X_test = testing_data.drop(columns=[target_column])
    y_test = testing_data[target_column]

    # Initialize XGBoost regressor
    reg = xgb.XGBRegressor(n_estimators=1000)

    # Fit XGBoost model to training data
    reg.fit(X_train, y_train)

    # Make predictions on testing data
    y_pred = reg.predict(X_test)

    # Add predicted values to results DataFrame
    results_df[predicted_column_name] = np.nan
    results_df.iloc[-len(y_pred):, results_df.columns.get_loc(predicted_column_name)] = y_pred

    # Plot actual, predicted, and training data
    plt.figure(figsize=(10, 6))

    # Plot training data
    plt.plot(training_data.index, training_data[target_column], color='#6E9ECF', label='Training Data')

    # Plot testing data
    plt.plot(testing_data.index, testing_data[target_column], color='orange', label='Testing Data')

    # Plot predicted values
    plt.plot(testing_data.index, y_pred, color='red', linestyle='--',label='Predicted Values')

    plt.xlabel('Index')
    plt.ylabel('Target Variable')
    plt.title(plot_title)
    plt.legend()
    plt.show()

    # Calculate evaluation metrics
    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100

    # Return evaluation metrics
    evaluation_metrics = {'Mean Squared Error': mse, 'Mean Absolute Error': mae, 'Mean Absolute Percentage Error': mape}
    return evaluation_metrics, xgb.plot_importance(reg, title=target_column, xlabel='Feature Importance Score', ylabel='Features', grid=False)


'''
apply XGboost function to predict calls 
'''
xgboost_model(train, test, 'calls', results_df, 'Predicted calls XGboost', 'Predicted calls XGboost')



'''
apply XGboost function to predict sickness rate 
'''
xgboost_model(train, test, 'sickness rate', results_df, 'Predicted sickness rate XGboost', 'Predicted sickness rate XGboost')


'''
preparing data for Prophet model 

'''

#create new data frame for calls
df_calls = df.reset_index()[["date", "calls"]].rename(
    columns={"date": "ds", "calls": "y"}
)
#splitting calls data into traing and testing data  
train_calls=df_calls.iloc[:-230]
test_calls=df_calls.iloc[-230:]

#create new data frame for sickness rate
df_sickness_rate = df.reset_index()[["date", "sickness rate"]].rename(
    columns={"date": "ds", "sickness rate": "y"}
)
#splitting sickness rate data into traing and testing data 
train_sickness_rate=df_sickness_rate.iloc[:-230]
test_sickness_rate=df_sickness_rate.iloc[-230:]


'''
creating function for Prophet model
'''

def prophet_model(training_data, testing_data, results_df, predicted_column_name, plot_title):
    # Initialize Prophet model
    model = Prophet(daily_seasonality=True)
    
    # Fit the model using training data
    model.fit(training_data)
    
    # Make future dataframe for testing data
    future = model.make_future_dataframe(periods=len(testing_data))
    
    # Predict values for testing data
    forecast = model.predict(future)
    
    # Extract last 230 predicted values
    predicted_values = forecast['yhat'].tail(230)
    
    # Add the last 230 predicted values to results_df
    results_df[predicted_column_name] = predicted_values.values
    
    # Calculate evaluation metrics
    mse = mean_squared_error(testing_data['y'], predicted_values)
    mae = mean_absolute_error(testing_data['y'], predicted_values)
    mape = np.mean(np.abs((testing_data['y'] - predicted_values) / testing_data['y'])) * 100
    
    # Print evaluation metrics
    print("Evaluation Metrics:")
    print("Mean Squared Error (MSE):", mse)
    print("Mean Absolute Error (MAE):", mae)
    print("Mean Absolute Percentage Error (MAPE):", mape)
    
  
    
    # Plot forecast
    model.plot(forecast, xlabel='Date', ylabel='Value')
    plt.title(plot_title)
    plt.show()



'''
predicting cals using prophet function
'''
prophet_model(train_calls, test_calls, results_df, 'Predicted calls Prophet', 'Calls Prediction with Prophet')


'''
predicting sickness rate using prophet function
'''

prophet_model(train_sickness_rate, test_sickness_rate, results_df, 'Predicted sickness rate Prophet', 'Predicted sickness rate Prophet')


'''
calculating Sby_drivers according to predicted values calls sickness rate
n_sby=(n_calls/(AHC_target ))+n_duty (sickness rate-1)

'''


AHC_target=5

results_df['Sby_xgboost']= (results_df['Predicted calls XGboost']/AHC_target)+results_df['n_duty']*(results_df['Predicted sickness rate XGboost']-1)
results_df['Sby_xgboost'] = results_df['Sby_xgboost'].clip(lower=0)
results_df['Sby_xgboost'] = results_df['Sby_xgboost'].round(0)
results_df['dafted_Xgboost']=results_df['sby_need'] - results_df['Sby_xgboost']
results_df['dafted_Xgboost']= results_df['dafted_Xgboost'].clip(lower=0)

results_df['Sby_ARIMA']= (results_df['Predicted calls Arima']/AHC_target)+results_df['n_duty']*(results_df['Predicted sickness rate Arima']-1)
results_df['Sby_ARIMA'] = results_df['Sby_ARIMA'].clip(lower=0)
results_df['Sby_ARIMA'] = results_df['Sby_ARIMA'].round(0)
results_df['dafted_ARIMA']=results_df['sby_need'] - results_df['Sby_ARIMA']
results_df['dafted_ARIMA']= results_df['dafted_ARIMA'].clip(lower=0)


results_df['Sby_Prophet']= (results_df['Predicted calls Prophet']/AHC_target)+results_df['n_duty']*(results_df['Predicted sickness rate Prophet']-1)
results_df['Sby_Prophet'] = results_df['Sby_Prophet'].clip(lower=0)
results_df['Sby_Prophet'] = results_df['Sby_Prophet'].round(0)
results_df['dafted_Prophet']=results_df['sby_need'] - results_df['Sby_Prophet']
results_df['dafted_Prophet']= results_df['dafted_Prophet'].clip(lower=0)


'''
Visiualizing dafted using predoction models vs dafted actual 

'''

# Plotting dafted vs dafted_xgboost
plt.figure(figsize=(10, 6))
sns.lineplot(data=results_df[['dafted', 'dafted_Xgboost']])
plt.xlabel('Index')
plt.ylabel('Value')
plt.title('Actual dafted vs dafted_Xgboost')
plt.legend(labels=['Actual dafted', 'dafted_Xgboost'])
plt.show()

# Plotting dafted vs dafted_arima
plt.figure(figsize=(10, 6))
sns.lineplot(data=results_df[['dafted', 'dafted_ARIMA']])
plt.xlabel('Index')
plt.ylabel('Value')
plt.title('dafted vs dafted_ARIMA')
plt.legend(labels=['Actual dafted', 'dafted_ARIMA'])
plt.show()

# Plotting dafted vs dafted_prophet
plt.figure(figsize=(10, 6))
sns.lineplot(data=results_df[['dafted', 'dafted_Prophet']])
plt.xlabel('Index')
plt.ylabel('Value')
plt.title('dafted vs dafted_Prophet')
plt.legend(labels=['Actual dafted', 'dafted_Prophet'])
plt.show()


'''
# Create histograms for the 'dafted' and 'dafted_Xgboost' columns
'''


plt.figure(figsize=(10, 5))

plt.subplot(1, 2, 1)
results_df['dafted'].plot(kind='hist', bins=20, color='blue', edgecolor='black')
plt.title('Histogram of dafted')
plt.xlabel('Value')
plt.ylabel('Frequency')

plt.subplot(1, 2, 2)
results_df['dafted_Xgboost'].plot(kind='hist', bins=20, color='green', edgecolor='black')
plt.title('Histogram of dafted_Xgboost')
plt.xlabel('Value')
plt.ylabel('Frequency')

plt.tight_layout()
plt.show()

FileNotFoundError: [Errno 2] No such file or directory: 'sickness_table.csv'